# I. Normalize Text

### Nomalize ancent

In [1]:
import re
import json

# Replacement function 
def normalize_accented_word(text):
    pattern = re.compile(r'(\w*)([áàảãạấầẩẫậắằẳẵặéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ])(\w*)')
    with open('bank/tones.json', 'r') as file: 
        tone_map = json.load(file)
    def replace(match):
        accented_char = match.group(2)
        base_char, tone = tone_map[accented_char]
        return f"{match.group(1)}{base_char}{match.group(3)}{tone}"
    
    text = pattern.sub(replace, text)
    return text

# Example usage
text = "khuyến nghị ịt"
modified_text = normalize_accented_word(text)
print(modified_text)


khuyên1 nghi5 it5


### Normalize Number

In [2]:
import json

with open("bank/base_numbers.json", "r") as file:
    BASE_NUMBERS = {int(key): value for key, value in json.load(file).items()}

with open("bank/number_levels.json", "r") as file:
    NUMBER_LEVELS = {int(key): value for key, value in json.load(file).items()}


def convert_number_2_digits(num: int):
    if num in BASE_NUMBERS:
        return BASE_NUMBERS[num]
    
    tens = num // 10
    base = num % 10
    if base > 0:
        return f"{BASE_NUMBERS[tens]} mươi {BASE_NUMBERS[base]}"
    
    return f"{BASE_NUMBERS[tens]} mươi"


def convert_number_3_digits(num:int):
    if num == 0:
        return ""

    remainder = num % 100
    hundred = num // 100
    if remainder == 0:
        return f"{BASE_NUMBERS[hundred]} trăm"
    
    if remainder < 10:
        return f"{BASE_NUMBERS[num // 100]} trăm linh {BASE_NUMBERS[remainder]}"
    
    return f"{BASE_NUMBERS[hundred]} trăm {convert_number_2_digits(remainder)}"


def convert_number_string_to_vietnamese(num:int):
    if num == 0:
        return 'không'

    if num in BASE_NUMBERS:
        return BASE_NUMBERS[num]
    
    if num < 100:
        return convert_number_2_digits(num)
    
    result = convert_number_3_digits(num % 1000)
    current_level = None
    
    for current_level in NUMBER_LEVELS:
        next_level = current_level * 1000
        if num // (next_level) == 0:
            break

        result = convert_number_3_digits(num % (next_level) // current_level) + " " + NUMBER_LEVELS[current_level] + " " + result
        
    level_base = num // current_level    
    
    if level_base == 0:
        return result
    
    if level_base in BASE_NUMBERS:
        return  f'{BASE_NUMBERS[level_base]} {NUMBER_LEVELS[current_level]} {result}'
    
    if level_base  > 99:
        return f'{convert_number_3_digits(level_base)} {NUMBER_LEVELS[current_level]} {result}'
    
    if level_base > 11:
        return  f'{convert_number_2_digits(level_base)} {NUMBER_LEVELS[current_level]} {result}'

In [3]:
import re

def normalize_number(text: str) -> str:
    # Regular expression to match numbers
    pattern = r'\d+'
    
    # Use re.sub with a lambda to replace each number with its Vietnamese word equivalent
    replaced_text = re.sub(pattern, lambda x: convert_number_string_to_vietnamese(int(x.group())), text)
    
    return replaced_text

In [4]:
normalize_number("103, 123, 113, 221, 1009, 1109, 10101")

'một trăm linh ba, một trăm hai mươi ba, một trăm mười ba, hai trăm hai mươi mốt, một nghìn không trăm linh chín, một nghìn một trăm linh chín, mười nghìn một trăm linh một'

### Normalize unit

In [5]:
import re
import json

# Hàm chuyển đổi đơn vị đo lường về tiếng Việt
def normalize_unit(text):
    # Từ điển chứa các đơn vị đo lường và bản dịch sang tiếng Việt
    with open('bank/units.json', 'r', encoding='utf-8') as json_file:
        units_mapping = json.load(json_file)
    # Sort units by length in descending order to prevent partial replacements
    sorted_units = sorted(units_mapping.keys(), key=len, reverse=True)

    # Create a single regex pattern for all units
    pattern = r'\b(' + '|'.join(map(re.escape, sorted_units)) + r')\b'
    
    # Replacement function to map the matched unit to its Vietnamese equivalent
    def replace_unit(match):
        return units_mapping[match.group(0)]
    
    # Use re.sub with the compiled pattern and replacement function
    return re.sub(pattern, replace_unit, text)

# Example usage:
text = "5 kg và 10 m3 nước"
converted_text = normalize_unit(text)
print(converted_text)  # Output: "5 kí lô gam và 10 mét khối nước"


5 kí lô gam và 10 mét khối nước


### Normalize Acronym

In [6]:
import json

def normalize_acronym(text: str):
    with open('bank/acronyms.json', 'r', encoding='utf-8') as json_file:
        acronym_map = json.load(json_file)
        
    sorted_units = sorted(acronym_map.keys(), key=len, reverse=True)
    
    pattern = r'\b(' + '|'.join(map(re.escape, sorted_units)) + r')\b'
    
    def replace_unit(match):
        return acronym_map[match.group(0)]
    
    return re.sub(pattern, replace_unit, text)

text = 'đcsvn'
normalize_acronym(text)

'đảng cộng sản việt nam'

### Normalize letter

In [7]:
import json

def normalize_letter(text: str):
    with open('bank/letters.json', 'r', encoding='utf-8') as json_file:
        letter_map = json.load(json_file)
        
    # Sort units by length in descending order to prevent partial replacements
    sorted_units = sorted(letter_map.keys(), key=len, reverse=True)
    
    # Create a single regex pattern for all units
    pattern = r'\b(' + '|'.join(map(re.escape, sorted_units)) + r')\b'
    
    # Replacement function to map the matched unit to its Vietnamese equivalent
    def replace_unit(match):
        return letter_map[match.group(0)]
    
    # Use re.sub with the compiled pattern and replacement function
    return re.sub(pattern, replace_unit, text)

text = 'ang g a'
normalize_letter(text)

'ang gờ a'

### Normalize symbol

In [51]:
import json

with open('bank/symbols.json', 'r', encoding='utf-8') as json_file:
    SYMBOLS = dict(sorted(json.load(json_file).items(), key=lambda item: len(item[0]), reverse=True))
    
def normalize_symbol(text: str):
    pattern = r'([\s\S])(' + '|'.join(map(re.escape, SYMBOLS)) + r')([\s\S])'
    
    def replace_symbol(match):
        return (match.group(1) if match.group(1) == ' ' else match.group(1) + ' ') + SYMBOLS[match.group(2)] + (match.group(3) if match.group(3) == ' ' else match.group(3) + ' ')
    
    return re.sub(pattern, replace_symbol, text)

In [52]:
text = 'a<> b'
normalize_symbol(text)

'a khác b'

### Normalize dot and comma

In [10]:

def normalize_dot_and_comma(text):
    signs = {
        '.': ' chấm ',
        ',': ' phẩy ',
    }
    
    def replace_dot_and_comma(match):
        return match.group(1) + signs[match.group(2)] + match.group(3)
    
    # remove duplicate
    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'\,{2,}', ',', text)
    
    text = re.sub(r'(\S)([,\.])(\S)', replace_dot_and_comma, text)
    text = re.sub(r'(\S)([,\.])', r'\1 \2', text)
    text = re.sub(r'([,\.])(\S)', r'\1 \2', text)
    return text 


In [11]:
normalize_dot_and_comma('.... . , 1.2 2,3')

'. . , 1 chấm 2 2 phẩy 3'

### Normalize Same Phoneme

In [12]:
with open('bank/same_phonemes.json', 'r', encoding='utf-8') as json_file:
    SAME_PHONEMES = dict(sorted(json.load(json_file).items(), key=lambda item: len(item[0]), reverse=True))

def normalize_same_phoneme(text: str):
    
    # Create a single regex pattern for all symbols
    pattern = r'(' + '|'.join(map(re.escape, SAME_PHONEMES)) + r')'
    
    # Replacement function to map the matched symbol to its Vietnamese equivalent
    def replace_symbol(match):
        return SAME_PHONEMES[match.group(0)]
    
    # Use re.sub with the compiled pattern and replacement function
    return re.sub(pattern, replace_symbol, text)

text = 'kiều không giống ghép quan nghệ nghi'
normalize_same_phoneme(text)

'ciều không dống gép qan ngệ ngi'

### Normalize Date

In [14]:
with open('bank/date_prefixs.json', 'r', encoding='utf-8') as json_file:
    DATE_PREFIXS = sorted(json.load(json_file), key=len, reverse=True)

In [15]:
import re

def normalize_date_pattern1(text: str):
    # Date pattern 1
    date_pattern1 = r'(\b\w{0,4}\b)\s*([12][0-9]|3[01]|0?[1-9])\/(1[0-2]|0?[1-9])\/(\d{1,4})'  # Example: 11/12/2002

    def replace(match):
        prefix = match.group(1).strip()
        day = match.group(2)
        month = match.group(3)
        year = match.group(4)
        
        if prefix == 'ngày':
            return f'{prefix} {day} tháng {month} năm {year}'
        else:
            return f'{prefix + " " if prefix != "" else ""}ngày {day} tháng {month} năm {year}'

    return re.sub(date_pattern1, replace, text)


In [16]:
normalize_date_pattern1('ngày 11/12/2002')

'ngày 11 tháng 12 năm 2002'

In [17]:
normalize_date_pattern1('11/12/2002')

'ngày 11 tháng 12 năm 2002'

In [18]:
normalize_date_pattern1('trưa 11/12/2002')

'trưa ngày 11 tháng 12 năm 2002'

In [19]:
import re

def normalize_date_pattern2(text: str):
    # Date pattern 1
    date_pattern2 = r'(\b\w{0,4}\b)\s([12][0-9]|3[01]|0?[1-9])\-(1[0-2]|0?[1-9])\-(\d{1,4})'  # Example: 11/12/2002

    def replace(match):
        prefix = match.group(1).strip()
        day = match.group(2)
        month = match.group(3)
        year = match.group(4)
        
        if prefix == 'ngày':
            return f'{prefix} {day} tháng {month} năm {year}'
        else:
            return f'{prefix + " " if prefix != "" else ""}ngày {day} tháng {month} năm {year}'

    return re.sub(date_pattern2, replace, text)


In [20]:
normalize_date_pattern2('sáng 11-12-2002')

'sáng ngày 11 tháng 12 năm 2002'

In [21]:
normalize_date_pattern2('thứ hai 11-12-2002')

'thứ hai ngày 11 tháng 12 năm 2002'

In [22]:
import re

def normalize_date_pattern3(text: str):
    # Date pattern 3
    date_pattern3 = r'(\b\w{0,5}\b)\s*(0?[1-9]|1[0,1,2])[\/|\-](\d{4})' # Example: 12/2022 -> tháng 12 năm 2002

    def replace(match):
        prefix = match.group(1).strip()
        month = match.group(2)
        year = match.group(3)
        print(prefix)
        if prefix == 'tháng':
            return f'{prefix} tháng {month} năm {year}'
        else:
            return f'{prefix + " " if prefix != "" else ""}tháng {month} năm {year}'

    return re.sub(date_pattern3, replace, text)


In [23]:
normalize_date_pattern3('12-2002')

'tháng 12 năm 2002'

In [24]:
normalize_date_pattern3('tháng 12-2002')

tháng


'tháng tháng 12 năm 2002'

In [31]:
import re

def normalize_date_pattern4(text: str):
    # Date pattern 4
    prefixs = "|".join(DATE_PREFIXS)
    date_pattern4 = r'(' + prefixs + r')\s([12][0-9]|3[01]|0?[1-9])[\-|\/](1[0-2]|0?[1-9])[\-|\/](\d{1,4})'

    return re.sub(date_pattern4, r'\1 ngày \2 tháng \3 năm \4', text)


In [32]:
normalize_date_pattern4('chiều 11/12/2002')

'chiều ngày 11 tháng 12 năm 2002'

In [33]:
def normalize_date(text: str):
    text = normalize_date_pattern4(text)
    text = normalize_date_pattern1(text)
    text = normalize_date_pattern2(text)
    text = normalize_date_pattern3(text)
    return text

### Normalize all text

In [35]:
import re



# Function to replace numbers with Vietnamese words and add spaces around commas
def normalize_vietnamese_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Normalize Date
    text = normalize_date(text)
    
    # Replace numbers with words
    text = normalize_number(text)
    
    # Normalize Accent
    text = normalize_accented_word(text)
    
    # Handle punctuation (remove unnecessary symbols, keep meaningful punctuation)
    text = re.sub(r'[^a-zA-Z0-9\sđâăêôơư.,]', '', text)
    
    # Normalize dot and comma
    text = normalize_dot_and_comma(text)
    
    text = normalize_unit(text)
    
    text = normalize_same_phoneme(text)
    
    return [t for t in text.split(' ')]

In [36]:
text = 'Sáng nay 11/12/2002, lực... lượng cứu hộ đã mở rộng phạm vi tìm kiếm thêm 11122002 m về phía cánh đồng gần khu vực nhà ở của các hộ dân. Thêm hai nạn nhân vừa được tìm thấy, nâng tổng số người tử vong lên 32.'
print(normalize_vietnamese_text(text=text))

['sang1', 'nay', 'ngay2', 'mươi2', 'môt5', 'thang1', 'mươi2', 'hai', 'năm', 'hai', 'ngin2', 'không', 'trăm', 'linh', 'hai', ',', 'lưc5', '.', 'lương5', 'cưu1', 'hô5', 'đa4', 'mơ3', 'rông5', 'pham5', 'vi', 'tim2', 'ciêm1', 'thêm', 'mươi2', 'môt5', 'triêu5', 'môt5', 'trăm', 'hai', 'mươi', 'hai', 'ngin2', 'không', 'trăm', 'linh', 'hai', 'mét', 'vê2', 'phia1', 'canh1', 'đông2', 'gân2', 'khu', 'vưc5', 'nha2', 'ơ3', 'cua3', 'cac1', 'hô5', 'dân', '.', 'thêm', 'hai', 'nan5', 'nhân', 'vưa2', 'đươc5', 'tim2', 'thây1', ',', 'nâng', 'tông3', 'sô1', 'ngươi2', 'tư3', 'vong', 'lên', 'ba', 'mươi', 'hai', '.']


# II. Anayze Phoneme

In [54]:
alphabet = [
    'a', 'ă', 'â', 'b', 'c', 'd', 'đ', 'e', 'ê', 'g', 'h', 'i', 
    'k', 'l', 'm', 'n', 'o', 'ô', 'ơ', 'p', 'q', 'r', 's', 't', 
    'u', 'ư', 'v', 'x', 'y'
]

In [55]:
# Dyads which never take a final consonant
group_1 = [
    'ai', 'ao', 'au', 'ay', 'âu', 'ây', 'eo', 'êu', 'iu', 
    'ia', 'oi', 'ôi', 'ơi', 'ua', 'ui', 'ưa', 'ưi', 'ưu'
]

In [56]:
# Dyads that optional need a final constant
group_2= [
    'oa', 'oe', 'uê', 'ươ', 'uy' 
]

In [57]:
# Dyads that always need a final constant
group_3 = [
    'a', 'ă', 'â', 'e', 'ê', 'i', 'o', 'ô', 'ơ', 'u', 'ư', 'iê', 'oă', 'oo', 'uâ', 'uô', 'ươ', 'uyê'
]

In [58]:
# Triads
group_4 = [
    'iêu', 'oai', 'oao', 'oay', 'oeo', 'uây', 'uôi', 'uya', 
    'uyu', 'ươi', 'ươu'
]
# Note: 'uyê' q|uyen

In [59]:
final_consonants = [
    'c', 'ch', 'ng', 'nh', 'm', 'n', 'p', 't'
]

In [60]:
head_consonants = [
    'b', 'c', 'd', 'đ', 'g', 'h', 'l', 'm', 'n', 'p', 'q', 
    'r', 's', 't', 'v', 'x', 'ch', 'kh', 'ng', 'nh', 'ph', 'th', 'tr' 
]

In [61]:
accent_list = ['1', '2', '3', '4', '5']

In [62]:
phoneme_bank = alphabet + group_1 + group_2+ head_consonants + \
            [char1 + char2 for char1 in group_2 for char2 in final_consonants] + \
            [char1 + char2 for char1 in group_3 for char2 in final_consonants] + accent_list + [',', '.']

phoneme_bank.sort(key=len, reverse=True)

In [63]:
len(phoneme_bank)

266

In [64]:
print(phoneme_bank)

['uyêch', 'uyêng', 'uyênh', 'oach', 'oang', 'oanh', 'oech', 'oeng', 'oenh', 'uêch', 'uêng', 'uênh', 'ươch', 'ương', 'ươnh', 'uych', 'uyng', 'uynh', 'iêch', 'iêng', 'iênh', 'oăch', 'oăng', 'oănh', 'ooch', 'oong', 'oonh', 'uâch', 'uâng', 'uânh', 'uôch', 'uông', 'uônh', 'ươch', 'ương', 'ươnh', 'uyêc', 'uyêm', 'uyên', 'uyêp', 'uyêt', 'oac', 'oam', 'oan', 'oap', 'oat', 'oec', 'oem', 'oen', 'oep', 'oet', 'uêc', 'uêm', 'uên', 'uêp', 'uêt', 'ươc', 'ươm', 'ươn', 'ươp', 'ươt', 'uyc', 'uym', 'uyn', 'uyp', 'uyt', 'ach', 'ang', 'anh', 'ăch', 'ăng', 'ănh', 'âch', 'âng', 'ânh', 'ech', 'eng', 'enh', 'êch', 'êng', 'ênh', 'ich', 'ing', 'inh', 'och', 'ong', 'onh', 'ôch', 'ông', 'ônh', 'ơch', 'ơng', 'ơnh', 'uch', 'ung', 'unh', 'ưch', 'ưng', 'ưnh', 'iêc', 'iêm', 'iên', 'iêp', 'iêt', 'oăc', 'oăm', 'oăn', 'oăp', 'oăt', 'ooc', 'oom', 'oon', 'oop', 'oot', 'uâc', 'uâm', 'uân', 'uâp', 'uât', 'uôc', 'uôm', 'uôn', 'uôp', 'uôt', 'ươc', 'ươm', 'ươn', 'ươp', 'ươt', 'ai', 'ao', 'au', 'ay', 'âu', 'ây', 'eo', 'êu', 'iu'

In [65]:
import re

def word2vec(word:str):
    v = [0] * len(phoneme_bank)
    for idx in range(0, len(phoneme_bank)):
        if word == '':
            return v
        if phoneme_bank[idx] in word:
            v[idx] = 1
            # print(f"Found {phoneme_bank[idx]} in {word}")
            word = re.sub(phoneme_bank[idx], '', word)
            # print(word)
    return v

In [66]:
text = 'Sáng nay, lực lượng cứu hộ đã mở rộng phạm vi tìm kiếm thêm 1010000101 m về phía cánh đồng gần khu vực nhà ở của các hộ dân. Thêm hai nạn nhân vừa được tìm thấy, nâng tổng số người tử vong lên 32.'
words = normalize_vietnamese_text(text=text)
print(words)
for word in words:
    print(f"{word}: {sum(word2vec(word))}")

NameError: name 'normalize_vietnamese_text' is not defined

In [67]:
print(f"quyên : {sum(word2vec('quyên'))}")

quyên : 2


In [68]:
import json

with open("phonemes.json", "w") as file:
    json.dump(phoneme_bank, file, indent='\t')

In [69]:
len(phoneme_bank)

266